In [2]:
%matplotlib notebook
import itertools
from pylab import *
import pandas as pd
from sklearn.decomposition import PCA
import yaml

In [25]:
# Used to get nearest neighbors for surface normals
depthXY_to_cloudXYZ = {}

# Converts lidar point cloud csv to depth matrix

df = pd.read_csv("../1504892561136210918-cloudpoint.csv", 
                 names = ['x', 'y', 'z', 'intensity', 'ring', 'rotation', 'revolution'])

df['distance'] = np.sqrt(df['x']**2 + df['y']**2 + df['z']**2)
# np.arctan2 is element-wise arc tangent that chooses the quadrant correctly
df['rotation-angle'] = np.arctan2(df['x'], df['y']) + np.pi
df['inclination-angle'] = np.arctan2(df['z'], df['distance'])

# img_height is number of lasers
# img_width is number of pts/number of lasers
img_height = 80
img_width = 2088

# calculate histogram for inclination angle
df['inclination-angle'] = np.arctan2(df['z'], df['distance'])
df_inclination_arr = df['inclination-angle'].as_matrix()
hist, bin_edges = np.histogram(df_inclination_arr, bins = img_height - 1)

df.sort_values(['rotation-angle'], inplace = True)

bin_size = 64

depth_img = np.zeros((img_width, img_height))

for i in range(img_width):
    df_rot_slice = df.iloc[bin_size * i : bin_size * (i + 1)].copy()
    df_rot_slice_arr = df_rot_slice[['x', 'y', 'z', 'distance']]
    pixel_bin_arr = np.digitize(df_rot_slice['inclination-angle'], bin_edges)

    for j in range(len(pixel_bin_arr)):
        pixel_vert_loc = pixel_bin_arr[j] - 1
        depth_img[i][pixel_vert_loc] = df_rot_slice_arr['distance'].iloc[j]
        cloudXYZ = (df_rot_slice_arr['x'].iloc[j], df_rot_slice_arr['y'].iloc[j], 
                    df_rot_slice_arr['z'].iloc[j])
        depthXY_to_cloudXYZ[(i, pixel_vert_loc)] = cloudXYZ

# Converts depth matrix to surface normal matrix

# image dimensions in NumPy array,
# 3rd dimension is for each component of normal vector
normal_img = np.zeros((img_width, img_height, 3))
# number of nearest point neighbors
# i.e. radius of 1 represents a 3x3 grid of nearest pixels for the middle pixel
nn_radius_close = 2
nn_radius_far = 2
# the following var are automatically set in algorithm
nn_radius = 0
nn_radius_for_loop = max(nn_radius_close, nn_radius_far)


for x in range(nn_radius_for_loop, depth_img.shape[0] - nn_radius_for_loop):
    for y in range(nn_radius_for_loop, depth_img.shape[1] - nn_radius_for_loop):
        if not depth_img[x][y]:
            continue
            
        # logic used for diff nn radius for close and far
        if depth_img[x][y] <= 20:
            nn_radius = nn_radius_close
        else:
            nn_radius = nn_radius_far
        
        # used to check for depth discontinuities
        curr_depth = depth_img[x][y]
        nn_arr = []
        x_start = x - nn_radius
        y_start = y - nn_radius
        
        # find out nearest neighbors' depths
        for x_n in range(2 * nn_radius + 1):
            for y_n in range(2 * nn_radius + 1):
                x_coord = x_start + x_n
                y_coord = y_start + y_n
                                
                # if there is no depth discontinuity, add it
                if np.abs(depth_img[x_coord][y_coord] - curr_depth) < .2:
                    if (x_coord, y_coord) in depthXY_to_cloudXYZ:
                        cloudXYZ_value = depthXY_to_cloudXYZ[(x_coord, y_coord)]
                        nn_arr.append(cloudXYZ_value)
        
        # can only do PCA if more than one nearest neighbor
        if (len(nn_arr) <= 1):
            continue
        nn_matrix = np.array(nn_arr)
               
        # mean center the data
        nn_matrix -= np.mean(nn_matrix, axis=0)
        # calculate the covariance matrix
        cov_matrix = np.cov(nn_matrix, rowvar=False)
        # calculate eigenvectors & eigenvalues of the covariance matrix
        # use 'eigh' rather than 'eig' since R is symmetric, 
        # the performance gain is substantial
        # eigenvectors are in increasing order of eigenvalue
        eigvals, eigvecs = np.linalg.eigh(cov_matrix)
        # sort eigenvectors in decreasing order
        # eigvecs = np.flip(eigvecs, axis=1)
        
        # the eigenvector corresponding to the smallest eigenvalue
        # represents the surface normal of the regression plane
        normal = eigvecs[:,0]
        
        # if surface normal is in the wrong direction, flip it!
        if np.dot(normal, np.array([0, 0, 0]) - 
                  np.array(list(depthXY_to_cloudXYZ[(x, y)]))) <= 0:
            normal = -normal
        
        normal_img[x][y][0] = normal[0]
        normal_img[x][y][1] = normal[1]
        normal_img[x][y][2] = normal[2] 

# Interpolation

mapping = {}
# number of nearest point neighbors
# i.e. radius of 1 represents a 3x3 grid of nearest pixels for the middle pixel
nn_radius = 1

for x in range(nn_radius, normal_img.shape[0] - nn_radius):
    for y in range(nn_radius, normal_img.shape[1] - nn_radius):
        # skip if not an empty pixel
        if normal_img[x][y][0] or normal_img[x][y][1] or normal_img[x][y][2]:
            continue

        nn_arr = []
        x_start = x - nn_radius
        y_start = y - nn_radius

        # find out nearest neighbors' surface normal vectors
        for x_n in range(2 * nn_radius + 1):
            for y_n in range(2 * nn_radius + 1):
                pixel = normal_img[x_start + x_n][y_start + y_n]
                if pixel[0] or pixel[1] or pixel[2]:
                    nn_arr.append(pixel)

        # skip sky
        if (x < img_height/3):
            if (len(nn_arr) <= 2):
                continue
        else:
            if (len(nn_arr) <= 0):
                continue
        
        nn_matrix = np.array(nn_arr)

        # fill empty pixel with average of filled in neighbors
        nn_avg = np.mean(nn_matrix, axis=0)
        mapping[(x, y)] = nn_avg
        
for x, y in mapping: 
    normal_img[x][y][0] = mapping[(x, y)][0]
    normal_img[x][y][1] = mapping[(x, y)][1]
    normal_img[x][y][2] = mapping[(x, y)][2]

In [26]:
lidar_table = np.zeros(normal_img.shape)
for key in depthXY_to_cloudXYZ:
    point = depthXY_to_cloudXYZ[key]
    if key[1] < lidar_table.shape[1] and key[0] < lidar_table.shape[0]:
        lidar_table[key[0], key[1]] = point

In [27]:
normal_ply_file_name = "normals.ply"
normal_colors = normal_img.reshape((normal_img.shape[0] * normal_img.shape[1], 3))
normal_points = lidar_table.reshape((lidar_table.shape[0] * lidar_table.shape[1], 3))
with open(normal_ply_file_name, 'w') as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(len(normal_points)))
    f.write("property float32 x\n")
    f.write("property float32 y\n")
    f.write("property float32 z\n")
    f.write("property float32 nx\n")
    f.write("property float32 ny\n")
    f.write("property float32 nz\n")
    f.write("end_header\n")
    for i in range(len(normal_points)):
        normal = normal_colors[i]
        f.write("{} {} {} {} {} {}\n".format(normal_points[i][0], normal_points[i][1], normal_points[i][2],
                                             normal[0], normal[1], normal[2]))

In [111]:
class Cluster:
    def __init__(self, normal, num):
        self.count = 1
        self.normal = normal
        self.id = num
        
    def add_normal(self, normal):
        self.normal = (self.normal * self.count + normal) / (self.count + 1)
        self.count += 1
        
    def remove_normal(self, normal):
        self.normal = (self.normal * self.count - normal) / (self.count - 1)

In [116]:
cluster_assignments = [ [Cluster(normal_img[i, j], i * normal_img.shape[1] + j) 
                         for j in range(normal_img.shape[1])]
                       for i in range(normal_img.shape[0]) ]
id_to_cluster = []
for row in cluster_assignments:
    for cluster in row:
        id_to_cluster.append(cluster)

cluster_id_to_coords = [[(i // normal_img.shape[1], i % normal_img.shape[1])]
                        for i in range(normal_img.shape[0] * normal_img.shape[1])]

to_check = set()
for i in range(normal_img.shape[0]):
    for j in range(1, normal_img.shape[1] - 1):
        if not np.array_equal(normal_img[i, j], np.array([0, 0, 0])):
            to_check.add((i, j))
            
while len(to_check) > 0:
    merges = dict()
    remove_set = set()
    
    for coord in to_check:
        x = coord[0]
        y = coord[1]
        
        cluster = cluster_assignments[x][y]
        normal = cluster.normal
        new_cluster = None
        
        should_remove = True
        for dx in range(-1, 2):
            for dy in range(-1, 2):
                new_x = x + dx
                if new_x >= normal_img.shape[0]:
                    new_x -= normal_img.shape[0]
                elif new_x < 0:
                    new_x += normal_img.shape[0]
                new_y = y + dy
                
                temp_cluster = cluster_assignments[new_x][new_y]
                if temp_cluster.id != cluster.id:
                    merge = (min(temp_cluster.id, cluster.id), max(temp_cluster.id, cluster.id)) 
                    if merge not in merges:
                        angle = np.arccos(np.dot(normal, temp_cluster.normal) / 
                                          np.linalg.norm(normal) / np.linalg.norm(temp_cluster.normal))
                        diff = lidar_table[new_x, new_y] - lidar_table[x, y]
                        dist = np.dot(diff, diff)
                        if angle < np.pi / 6 and dist < 1.0:
                            should_remove = False
                            merges[merge] = angle
        
#         if should_remove:
#             remove_set.add(coord)
    print(len(merges))
    if len(merges) == 0:
        break
        
    potential_merges = sorted(merges.keys(), key = lambda x: x[0])
    current_cluster = potential_merges[0][0]
    min_merge_candidate = potential_merges[0][1]
    min_merge_angle = np.pi
    for i in range(1, len(potential_merges) + 1):
        if i == len(potential_merges) or potential_merges[i][0] != current_cluster:
            coords = cluster_id_to_coords[current_cluster]
            cluster_id_to_coords[current_cluster] = []
            new_cluster = id_to_cluster[min_merge_candidate]
            for coord in coords:
                new_cluster.add_normal(normal_img[coord[0]][coord[1]])
                cluster_assignments[coord[0]][coord[1]] = new_cluster
            cluster_id_to_coords[min_merge_candidate] += coords
            min_merge_angle = np.pi
            if i < len(potential_merges) - 1:
                min_merge_candidate = potential_merges[i + 1][1]
                current_cluster = potential_merges[i + 1][0]
        else:
            if merges[potential_merges[i]] < min_merge_angle:
                min_merge_candidate = potential_merges[i][1]
                min_merge_angle = merges[potential_merges[i]]
            
#     to_check = to_check.difference(remove_set)
        

/Users/garychen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
/Users/garychen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in arccos


307046
25642
12027
6038
3086
1584
826
447
236
127
62
31
15
10
8
5
2
1
0


In [117]:
import random

blue = np.array([0, 0, 1])
red = np.array([1, 0, 0])
ids = []
for row in cluster_assignments:
    for cluster in row:
        ids.append(cluster.id)
max_id = max(ids)
colors = [(random.random(), random.random(), random.random())for i in ids]
with open("cluster_points.csv", 'w') as f1, open("cluster_colors.csv", 'w') as f2:
    lidars = lidar_table.reshape((lidar_table.shape[0] * lidar_table.shape[1], 3))
    
    for i in range(len(ids)):
        f1.write("{}, {}, {}\n".format(lidars[i][0], lidars[i][1], lidars[i][2]))
        color = colors[ids[i]]
        f2.write("{}, {}, {}\n".format(color[0], color[1], color[2]))